In [7]:
print("Please check to create the DB with Beev Electic Vehicle Specs Data")
print("Alex, please answer")

Please check to create the DB with Beev Electic Vehicle Specs Data
Alex, please answer


In [10]:
#LIBRARIES
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#Graphics
import matplotlib.pyplot as plt
import seaborn as sns
#Scalers
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#Data selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import PCA
#ML Classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
#METRICS
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score,accuracy_score
from sklearn.metrics import mean_squared_error
#RANDOM
import random as rd
#ML REGRESSION
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor

In [11]:
df = pd.read_csv("Beev Electric Vehicle Specs Data.csv")

In [12]:
df

,Full Name,Model,Brand,Acceleration,Range,Average Winter Range,Highway Winter Range,City Winter Range,Average Summer Range,Highway Summer Range,...,Type of Driver,Max Speed,Max Trunk Volume (mm),Min Trunk Volume (mm),youtube_video,Creation Date,Modified Date,Slug,Creator,unique id
0,Aiways U5,U5,Aiways,7.5,410.0,285.0,235.0,340.0,385.0,300.0,...,NaN,150.0,1555.0,432.0,NaN,"Sep 19, 2021 20:50","Oct 20, 2021 15:48",NaN,(App admin),1632077402600x431044176816571400
1,Audi e-tron 50 quattro,e-tron 50,Audi,6.8,280.0,240.0,205.0,285.0,320.0,255.0,...,1.0,190.0,1725.0,660.0,NaN,"Jul 20, 2020 15:44","Nov 16, 2021 10:24",NaN,(deleted thing),1595252663946x895911395902033000
2,Audi e-tron 55 quattro,e-tron 55,Audi,5.7,437.0,315.0,265.0,370.0,410.0,330.0,...,1.0,200.0,1725.0,660.0,NaN,"Jul 20, 2020 11:53","Nov 16, 2021 10:29",NaN,(deleted thing),1595238801413x563796210467635700
3,Audi e-tron GT quattro,e-tron GT quattro,Audi,4.1,487.0,355.0,305.0,405.0,475.0,390.0,...,NaN,245.0,NaN,405.0,NaN,"Sep 19, 2021 22:07","Nov 16, 2021 10:37",NaN,(App admin),1632082037901x768751924168235500
4,Audi e-tron GT RS,e-tron GT RS,Audi,3.3,472.0,345.0,295.0,395.0,460.0,375.0,...,NaN,250.0,NaN,366.0,NaN,"Sep 19, 2021 22:13","Nov 16, 2021 10:41",NaN,(App admin),1632082391017x236732476503401120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,Volkswagen ID.5GTX,ID.5 GTX,Volkswagen,6.3,410.0,345.0,296.0,400.0,465.0,375.0,...,NaN,180.0,1561.0,549.0,NaN,"Nov 10, 2021 10:58","Dec 6, 2021 10:21",NaN,(App admin),1636538310300x358194465958965760
146,Volkswagen Transporter 6.1 ABTe,Transporter 6.1 ABTe,Volkswagen,17.4,138.0,NaN,NaN,NaN,NaN,NaN,...,NaN,130.0,NaN,NaN,NaN,"Sep 25, 2021 17:08","Oct 18, 2021 12:51",NaN,(App admin),1632582499541x281936676783606460
147,Volvo XC40 Recharge Twin,XC40 Recharge Twin,Volvo,4.9,400.0,315.0,270.0,365.0,440.0,355.0,...,1.0,180.0,1373.0,413.0,NaN,"Jun 3, 2020 14:40","Nov 15, 2021 19:04",NaN,NaN,1591188039290x449226619630125060
148,Volvo C40 Recharge Twin,C40 Recharge,Volvo,4.7,420.0,290.0,240.0,350.0,380.0,300.0,...,NaN,180.0,1205.0,419.0,NaN,"Sep 19, 2021 23:00","Oct 18, 2021 15:26",NaN,(App admin),1632085219484x265759528215476100


In [15]:
nan_values = df.isna().sum()
nan_values.head(25)

Full Name                                  0
Model                                      0
Brand                                      0
Acceleration                               8
Range                                      1
Average Winter Range                      14
Highway Winter Range                      15
City Winter Range                         15
Average Summer Range                      14
Highway Summer Range                      15
City Summer Range                         16
Optimal Slow Charge                        8
Optimal Fast Charge                       19
Battery Capacity                           0
Useable Battery Capacity                   3
Category                                   0
Government Incentive Category for Help    13
Vehicle Consumption                       26
Average Winter Consumption                19
Highway Winter Consumption                19
City Winter Consumption                   19
Average Summer Consumption                18
Highway Su